<a href="https://colab.research.google.com/github/adalbertii/Modele-regresyjne/blob/main/neural_network_pytorch_regression_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobranie niezbędnych bibliotek

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
print(data.feature_names)
X, y = data.data, data.target

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[:5]

In [ ]:
y[:5]

Budowa modelu

In [2]:
import torch.nn as nn
# definicja modelu
model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [ ]:
model

Ustalenie funkcji straty oraz algorytmu optymalizacyjnego

In [12]:

import torch.optim as optim
# ustalenie funkcji straty oraz metody optymalizacji
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Proces uczenia modelu

In [13]:
import copy
import numpy as np
import torch
import tqdm
from sklearn.model_selection import train_test_split

# podział zbioru danych na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [14]:
# ustalenie wartości hiperparametrów
n_epochs = 20   # number of epochs to run
batch_size = 10  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)



In [15]:
# inicjalizacja wartości parametrów modelu
best_mse = np.inf   # nieskończoność
best_weights = None
history = []



In [ ]:
# pętla treningowa modelu
for epoch in range(n_epochs):
    print('Epoka: ', epoch)
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:

            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]

            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            # aktualizacja wag
            optimizer.step()

            bar.set_postfix(mse=float(loss))
    # weryfikacja parametru 'accuracy' na końcu każdej epoki
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())



In [ ]:
best_weights

In [ ]:
# zaldowanie do modelu wyuczone wagi
model.load_state_dict(best_weights)

Weryfikacja modelu

In [ ]:
import matplotlib.pyplot as plt
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()

Kompletny proces z uwzględnieniem standaryzacji danych wejściowych



In [20]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# pobranie danych
data = fetch_california_housing()
X, y = data.data, data.target

# podział danych na dane treningowe i testowe
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)



In [ ]:
X_train[:10]

In [22]:
# standaryzacja danych
scaler = StandardScaler()
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)



In [ ]:
X_test[:10]

In [24]:
#kowersja do tensora 2D(wymóg biblioteki pyTorch )
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)


In [ ]:
X_test

In [26]:

# definicja modelu
model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

# definicja funkcji straty ora metody optymalizatora
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 30
batch_size = 10
batch_start = torch.arange(0, len(X_train), batch_size)



In [ ]:
# trening modelu
best_mse = np.inf
best_weights = None
history = []

for epoch in range(n_epochs):
    print('Epoka: ', epoch)
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:

            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]

            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            # aktualizacja wag
            optimizer.step()

            bar.set_postfix(mse=float(loss))
    # ewaluacja modelu po każdej epoce
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())



In [ ]:
# załadowanie modelu wytrenowanymi wagami
model.load_state_dict(best_weights)
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()



In [ ]:
model.eval()
with torch.no_grad():
    # predykcja na danych testowych (5 przykładow)
    for i in range(5):
        X_sample = X_test_raw[i: i+1]
        X_sample = scaler.transform(X_sample)
        X_sample = torch.tensor(X_sample, dtype=torch.float32)
        y_pred = model(X_sample)
        print(f"{X_test_raw[i]} -> {y_pred[0].numpy()} (expected {y_test[i].numpy()})")


In [ ]:
model.eval()
with torch.no_grad():
    # predykcja na danych testowych (50 przykładow)
    for i in range(50):
        X_sample = X_test_raw[i: i+1]
        X_sample = scaler.transform(X_sample)
        X_sample = torch.tensor(X_sample, dtype=torch.float32)
        y_pred = model(X_sample)
        print(f"{y_pred[0].numpy()} (expected {y_test[i].numpy()})")